In [1]:
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

In [2]:
train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

In [23]:
#LOOCV increasing the labeled training set with separate test set
X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

print(X_train.shape)
print(y_train.shape)

# X_all = X_train.append(X_test)
# y_all = y_train.append(y_test)

groups = train_df['name']

outcomes = []

clf = RandomForestClassifier(criterion='gini',
                             max_features=None,
                             min_samples_split=5,
                             n_estimators=100)

print(len(groups))
def run_pseudo_labeling(clf):
    logo = LeaveOneGroupOut()
    group = 0
    
    labeled_index = np.empty(0, int)
    unlabeled_index = np.empty(0, int)
    
    percentage = 0.1
    
    for train_index, test_index in logo.split(X_train, groups=groups):
        labeled_index = np.append(labeled_index, test_index)
        unlabeled_index = np.array(list(set(X_train.index.values).symmetric_difference(labeled_index)))
        
        X_labeled = X_train.values[labeled_index]
        y_labeled = y_train.values[labeled_index]
        
        print(len(X_labeled))

        clf.fit(X_labeled, y_labeled)
        predictions = clf.predict(X_test)
        
        print(classification_report(y_test, predictions))
        print(confusion_matrix(y_test, predictions))
        
        accuracy = balanced_accuracy_score(y_test, predictions)
#         outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    
        # Predict and add unlabeled samples to training set
#        predictions = clf.predict(X_train.values[unlabeled_index])
        # Predict and add unlabeled data with highest probability to training set
        pred_probs = clf.predict_proba(X_train.values[unlabeled_index])
        pred_labels = [np.argmax(e) for e in pred_probs]
        max_pred_probs = [np.max(e) for e in pred_probs]
        
#         preds = clf.predict(X_train.values[unlabeled_index])
#         print('actual labels')
#         sum = 0
#         for i in range(len(preds)):
#             if preds[i] > 1:
#                 print(preds[i])
#                 sum += 1
#         print(sum)
#         for i in range(len(y_train.values[unlabeled_index])):
#             print(y_train.values[unlabeled_index][i])
#         print('predicted labels')
#         sum = 0
#         for i in range(len(pred_labels)):
#             if pred_labels[i] > 1:
#                 print(pred_labels[i])
#                 sum += 1
#         print(sum)
        cat_pred_probs = [0 if e <= 1 else 10 for e in pred_labels]
        data_df = { 'probs': max_pred_probs, 'cat': cat_pred_probs, 'label': pred_labels }
        max_pred_probs_df = pd.DataFrame(data=data_df)
#        max_pred_probs_df = max_pred_probs_df.sort_values(by=['cat', 'probs'], ascending=False)
        max_pred_probs_df = max_pred_probs_df.sort_values(by=['probs'], ascending=False)
        print(max_pred_probs_df.head())
        
        pseudo_size = int(percentage * len(unlabeled_index))
        max_indices = max_pred_probs_df.index.values[:pseudo_size]
        
        X_pseudo = np.array(X_train.values[unlabeled_index])[max_indices]
        y_pseudo = np.array([np.argmax(e) for e in pred_probs])[max_indices]
        
        X_aug = X_labeled.copy()
        X_aug = np.append(X_aug, X_pseudo, axis=0)
        y_aug = y_labeled.copy()
        y_aug = np.append(y_aug, y_pseudo)
        
        clf.fit(X_aug, y_aug)
        predictions = clf.predict(X_test)
        
#        y_augmented = y_train.copy(deep=True)
#        y_augmented.values[unlabeled_index] = predictions
        
#        clf.fit(X_train, y_augmented)
#        predictions = clf.predict(X_test)
        
        print(classification_report(y_test, predictions))
        print(confusion_matrix(y_test, predictions))
        
        accuracy = balanced_accuracy_score(y_test, predictions)
#         outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    
        group += 1
#     mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_pseudo_labeling(clf)

(21418, 150)
(21418,)
21418
6093
              precision    recall  f1-score   support

           0       0.91      0.94      0.93      2566
           1       0.89      0.93      0.91      2661
           2       0.74      0.62      0.67       395
           3       0.82      0.52      0.64       411

   micro avg       0.89      0.89      0.89      6033
   macro avg       0.84      0.75      0.79      6033
weighted avg       0.88      0.89      0.88      6033

[[2414   93   17   42]
 [ 137 2487   33    4]
 [  75   77  243    0]
 [  20  141   37  213]]
Group 0 accuracy: 0.7522032329601237
      cat  label  probs
8921    0      0    1.0
4563    0      0    1.0
4588    0      0    1.0
4587    0      0    1.0
4586    0      0    1.0
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      2566
           1       0.88      0.93      0.91      2661
           2       0.70      0.59      0.64       395
           3       0.81      0.51      0.

IndexError: arrays used as indices must be of integer (or boolean) type

In [10]:
print(max_indices)

NameError: name 'max_indices' is not defined

In [9]:
#LOOCV pseudo-labeling with a separate test set
#sample_rate relative to validate set
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = pd.DataFrame(train_df['state'])

X_test = test_df.drop(['state', 'name'], axis=1)
y_test = pd.DataFrame(test_df['state'])

groups = train_df['name']

outcomes = []

def run_pseudo_labeling():
    logo = LeaveOneGroupOut()
    group = 0
    
    for train_index, test_index in logo.split(X_train, groups=groups): 

        X = pd.DataFrame(data=X_train.values[train_index])
        y = pd.DataFrame(data=y_train.values[train_index], columns=['state'])

        model = PseudoLabeler(
            RandomForestClassifier(criterion='gini',
                                   max_features=None,
                                   min_samples_split=5,
                                   n_estimators=100),
        #    GradientBoostingClassifier(n_estimators=100),
            pd.DataFrame(X_train.values[test_index]),
            'state',
            sample_rate=0.0
        )
        
        # Train the model and use it to predict
        model.fit(X, y)
        y_hat = model.predict(X_test)
        
        y_prob = model.predict_proba(X_test)
        print(y_prob)
        print(classification_report(y_test, y_hat))
        print(confusion_matrix(y_test, y_hat))        
        accuracy = balanced_accuracy_score(y_test, y_hat)
        print("Group {0} accuracy: {1}".format(group, accuracy))
        
        model.set_params(sample_rate=0.3)
        model.fit(X, y)
        y_hat = model.predict(X_test)
        print(y_prob)
        
        print(classification_report(y_test, y_hat))
        print(confusion_matrix(y_test, y_hat))        
        accuracy = balanced_accuracy_score(y_test, y_hat)
        outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    
        group += 1
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_pseudo_labeling()

/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


[[0.07890476 0.66990476 0.14735714 0.10383333]
 [0.06       0.65457143 0.1962619  0.08916667]
 [0.06533333 0.76040476 0.11309524 0.06116667]
 ...
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]]
              precision    recall  f1-score   support

           0       0.99      0.75      0.85      2566
           1       0.75      0.98      0.85      2661
           2       0.58      0.62      0.60       395
           3       0.79      0.31      0.45       411

   micro avg       0.81      0.81      0.81      6033
   macro avg       0.77      0.66      0.69      6033
weighted avg       0.84      0.81      0.81      6033

[[1922  554   58   32]
 [   1 2610   50    0]
 [  22  127  243    3]
 [   3  209   70  129]]
Group 0 accuracy: 0.6647296200881639


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:55: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


[[0.07890476 0.66990476 0.14735714 0.10383333]
 [0.06       0.65457143 0.1962619  0.08916667]
 [0.06533333 0.76040476 0.11309524 0.06116667]
 ...
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.         0.         0.        ]]
              precision    recall  f1-score   support

           0       0.99      0.70      0.82      2566
           1       0.72      0.98      0.83      2661
           2       0.57      0.64      0.60       395
           3       0.80      0.30      0.44       411

   micro avg       0.79      0.79      0.79      6033
   macro avg       0.77      0.66      0.67      6033
weighted avg       0.83      0.79      0.79      6033

[[1806  663   69   28]
 [   1 2607   51    2]
 [  13  128  253    1]
 [   2  217   69  123]]
Group 0 accuracy: 0.6558256132579678


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


KeyboardInterrupt: 

In [7]:
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import BaseEstimator, RegressorMixin
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

class PseudoLabeler(BaseEstimator, RegressorMixin):
    
    def __init__(self, model, test, target, sample_rate=0.2, seed=42):
        self.sample_rate = sample_rate
        self.seed = seed
        self.model = model
        self.model.seed = seed
        
        self.test = test
        self.target = target
        
    def get_params(self, deep=True):
        return {
            "sample_rate": self.sample_rate,
            "seed": self.seed,
            "model": self.model,
            "test": self.test,
            "target": self.target
        }
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, X, y):
        if self.sample_rate > 0.0:
            augmented_X_train, augmented_y_train = self.__create_augmented_train(X, y)
            self.model.fit(
                augmented_X_train,
                augmented_y_train
            )
        else:
            self.model.fit(X, y)
            
        return self
    
    def __create_augmented_train(self, X, y):
        '''
        Create and return the augmented_train set that consists
        of pseudo-labeled and labeled data.
        '''
        num_of_samples = int(len(self.test) * self.sample_rate)
        
        # Train the model and create the pseudo-labels
        self.model.fit(X, y)
        pseudo_labels = self.model.predict(self.test)
        
        # Add the pseudo-labels to the test set
        augmented_test = self.test.copy(deep=True)
        augmented_test[self.target] = pseudo_labels
        
        # Take a subset of the test set with pseudo-labels and append in onto
        # the training set
        sampled_test = augmented_test.sample(n=num_of_samples)
        temp_train = pd.concat([X, y], axis=1)
#         print(sampled_test.head())
#         print(temp_train.head())
        augmented_train = pd.concat([sampled_test, temp_train])
        augmented_train = shuffle(augmented_train)
        augmented_X = augmented_train.drop(['state'], axis=1)
        augmented_y = augmented_train['state']
        
        return augmented_X, augmented_y
    
    def predict(self, X):
        return self.model.predict(X)
    
    def predict_proba(self, X):
        return self.model.predict_proba(X)
    
    def get_model_name(self):
        return self.model.__class__.__name__

In [88]:
#Pseudo-labeling on actual test set
train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

# Preprocess the data
X_train, X_test = train_df.drop(['state', 'name'], axis=1), test_df.drop(['state', 'name'], axis=1)
y_train, y_test = pd.DataFrame(train_df['state']),  pd.DataFrame(test_df['state'])

# Create the PseudoLabeler with XGBRegressor as the base regressor
model = PseudoLabeler(
#    RandomForestClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100),
    X_test,
    'state',
    sample_rate=0.2
)

# Train the model and use it to predict
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

print(classification_report(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
accuracy = balanced_accuracy_score(y_test, y_hat)
print('balanced accuracy', accuracy)

(21418, 150)
(21418, 1)


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


            x0        y0        z0        x1        y1        z1        x2  \
5918  0.071289 -0.014648  1.001953  0.057617 -0.015625  0.992188  0.067383   
4354  0.079102 -0.003906  0.993164  0.070312 -0.004883  0.998047  0.075195   
3098  0.068359 -0.004883  1.000000  0.056641 -0.009766  0.990234  0.053711   
5199  0.055664 -0.009766  0.991211  0.055664 -0.013672  0.991211  0.064453   
2546  0.071289 -0.010742  0.990234  0.072266 -0.007812  0.991211  0.072266   

            y2        z2        x3  ...       x47       y47       z47  \
5918 -0.012695  0.990234  0.066406  ...  0.071289 -0.017578  1.000000   
4354 -0.013672  0.992188  0.076172  ...  0.082031 -0.008789  0.996094   
3098 -0.010742  0.983398  0.070312  ...  0.062500 -0.009766  0.986328   
5199 -0.011719  0.999023  0.066406  ...  0.066406 -0.014648  0.996094   
2546 -0.006836  0.991211  0.072266  ...  0.075195 -0.010742  0.985352   

           x48       y48       z48       x49       y49       z49  state  
5918  0.061523 -0.

In [ ]:
#Incremental pseudo-labeling
#Case 0: add pseudo-labeled data incrementally, no discrimination. We do per subject
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
import random

def create_augmented_train(model, X, y, X_test, sample_rate, target='state'):
    '''
    Create and return the augmented_train set that consists
    of pseudo-labeled and labeled data.
    '''
    num_of_samples = int(len(X_test) * sample_rate)

    # Train the model and create the pseudo-labels
    #model.fit(X, y)
    pseudo_labels = model.predict(X_test)

    # Add the pseudo-labels to the test set
    augmented_test = X_test.copy(deep=True)
    augmented_test[target] = pseudo_labels

    # Take a subset of the test set with pseudo-labels and append in onto
    # the training set
    sampled_test = augmented_test.sample(n=num_of_samples)
    temp_train = pd.concat([X, y], axis=1)
#         print(sampled_test.head())
#         print(temp_train.head())
    augmented_train = pd.concat([sampled_test, temp_train])
    augmented_train = shuffle(augmented_train)
    augmented_X = augmented_train.drop(['state'], axis=1)
    augmented_y = pd.DataFrame(data=augmented_train['state'], columns=['state'])
#     augmented_y = augmented_train['state']
    
    return augmented_X, augmented_y

train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

X_train, X_test = train_df.drop(['state', 'name'], axis=1), test_df.drop(['state', 'name'], axis=1)
y_train, y_test = pd.DataFrame(train_df['state']),  pd.DataFrame(test_df['state'])

groups_dict = train_df.groupby('name')

# Train initial model
index = np.random.randint(0,len(groups_dict), dtype='int')
subject_supervised = list(groups_dict.groups)[index]

# model = PseudoLabeler(
#     RandomForestClassifier(n_estimators=10, warm_start=True),
# #    GradientBoostingClassifier(n_estimators=100),
#     X_test,
#     'state',
#     sample_rate=0.0
# )

model = RandomForestClassifier(criterion='gini',
                             max_features=None,
                             min_samples_split=5,
                             n_estimators=100,
                             warm_start=True)

X_supervised = pd.DataFrame(data=X_train.values[groups_dict.groups[subject_supervised]])
y_supervised = pd.DataFrame(data=y_train.values[groups_dict.groups[subject_supervised]], columns=['state'])

model.fit(X_supervised, y_supervised)
y_hat = model.predict(X_test)

print('_______initial supervised model______')
print(classification_report(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
accuracy = balanced_accuracy_score(y_test, y_hat)
print('balanced accuracy', accuracy)

current_X = X_supervised.copy(deep=True)
current_y = y_supervised.copy(deep=True)

sample_rate = 0.2
for subject in groups_dict.groups:
    # The rest of the subjects used to augment training set
    if subject != subject_supervised:
        model.n_estimators += 10
        #y_hat = model.predict(X_train.values[groups_dict.groups[subject]])
        #model.set_params(test=X_train.values[groups_dict.groups[subject]])
        new_test = pd.DataFrame(data=X_train.values[groups_dict.groups[subject]])
        current_X, current_y = create_augmented_train(model, current_X, current_y, new_test, sample_rate)
        print(current_X.shape, current_y.shape)
        model.fit(current_X, current_y)
        y_hat = model.predict(X_test)
        print('_______', subject, '______')
        print(classification_report(y_test, y_hat))
        print(confusion_matrix(y_test, y_hat))
        accuracy = balanced_accuracy_score(y_test, y_hat)
        print('balanced accuracy', accuracy)
     

/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:70: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______initial supervised model______
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      2566
           1       0.87      0.94      0.90      2661
           2       0.74      0.58      0.65       395
           3       0.83      0.52      0.64       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.84      0.74      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2391  119   15   41]
 [ 137 2492   30    2]
 [  72   94  228    1]
 [  17  146   35  213]]
balanced accuracy 0.7409384685119196
(6876, 150) (6876, 1)


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______ daniel ______
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      2566
           1       0.87      0.94      0.90      2661
           2       0.73      0.57      0.64       395
           3       0.82      0.51      0.63       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.83      0.74      0.77      6033
weighted avg       0.88      0.88      0.88      6033

[[2386  124   15   41]
 [ 135 2494   29    3]
 [  73   92  227    3]
 [  17  145   39  210]]
balanced accuracy 0.7381814993704915
(7100, 150) (7100, 1)


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______ sue ______
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2566
           1       0.87      0.94      0.91      2661
           2       0.74      0.58      0.65       395
           3       0.82      0.52      0.63       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.84      0.74      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2388  122   15   41]
 [ 132 2496   30    3]
 [  72   90  230    3]
 [  17  146   36  212]]
balanced accuracy 0.7416795336525579
(7697, 150) (7697, 1)


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______ lenin ______
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2566
           1       0.87      0.94      0.91      2661
           2       0.73      0.58      0.65       395
           3       0.83      0.51      0.63       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.84      0.74      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2389  121   15   41]
 [ 127 2502   30    2]
 [  71   92  231    1]
 [  19  145   39  208]]
balanced accuracy 0.7405404807819318
(8600, 150) (8600, 1)


In [38]:
#Incremental pseudo-labeling
#Case 1: add pseudo-labeled data incrementally, most confident PTs have higher priority
from sklearn.model_selection import LeaveOneGroupOut
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
import random

# def create_augmented_train(model, X, y, X_test, sample_rate, target='state'):
#     '''
#     Create and return the augmented_train set that consists
#     of pseudo-labeled and labeled data.
#     '''
#     num_of_samples = int(len(X_test) * sample_rate)

#     # Train the model and create the pseudo-labels
#     #model.fit(X, y)
#     pseudo_labels = model.predict(X_test)

#     # Add the pseudo-labels to the test set
#     augmented_test = X_test.copy(deep=True)
#     augmented_test[target] = pseudo_labels

#     # Take a subset of the test set with pseudo-labels and append in onto
#     # the training set
#     sampled_test = augmented_test.sample(n=num_of_samples)
#     temp_train = pd.concat([X, y], axis=1)
# #         print(sampled_test.head())
# #         print(temp_train.head())
#     augmented_train = pd.concat([sampled_test, temp_train])
#     augmented_train = shuffle(augmented_train)
#     augmented_X = augmented_train.drop(['state'], axis=1)
#     augmented_y = pd.DataFrame(data=augmented_train['state'], columns=['state'])
# #     augmented_y = augmented_train['state']
    
#     return augmented_X, augmented_y

def create_augmented_data(model, X, y, X_train, sample_rate):
    print(len(X))
    print(len(y))
    '''
    Create and return the augmented_train set that consists
    of pseudo-labeled and labeled data.
    '''
#     X_pseudo = pd.DataFrame(data=X_train)
    # Create the pseudo-labels
    pred_probs = model.predict_proba(X_train)
    pred_labels = [np.argmax(e) for e in pred_probs]
    max_pred_probs = [np.max(e) for e in pred_probs]
    
    cat_pred_probs = [0 if e <= 1 else 10 for e in pred_labels]
    data_df = { 'probs': max_pred_probs, 'cat': cat_pred_probs }
    max_pred_probs_df = pd.DataFrame(data=data_df)
    max_pred_probs_df = max_pred_probs_df.sort_values(by=['probs'], ascending=False)
    print(max_pred_probs_df.head())
        
    pseudo_size = int(sample_rate * len(X_train))
    max_indices = max_pred_probs_df.index.values[:pseudo_size]
    
#     preds = model.predict(X_train)
#     y_pseudo = pd.DataFrame(data=preds, dtype=np.int8)

    X_pseudo = pd.DataFrame(data=X_train[max_indices])
    pred_labels = np.array(pred_labels)[max_indices]
    y_pseudo = pd.DataFrame(data=pred_labels, dtype=np.int8)

    # Add the pseudo-labels to the previous training set
    X_aug = pd.concat([X, X_pseudo], ignore_index=True)
    y_aug = pd.concat([y, y_pseudo], ignore_index=True)

    return X_aug, y_aug

train_df = pd.read_csv('../data/processed/train.csv')
test_df = pd.read_csv('../data/processed/test.csv')

X_train, X_test = train_df.drop(['state', 'name'], axis=1), test_df.drop(['state', 'name'], axis=1)
y_train, y_test = pd.DataFrame(train_df['state']),  pd.DataFrame(test_df['state'])

groups_dict = train_df.groupby('name')

# Train initial model
index = np.random.randint(0,len(groups_dict), dtype='int')
subject_supervised = list(groups_dict.groups)[index]
subjects_unsupervised = list()

for name in groups_dict.groups.keys():
    if name != subject_supervised:
        subjects_unsupervised.extend(groups_dict.groups[name].values)

model = RandomForestClassifier(criterion='gini',
                             max_features=None,
                             min_samples_split=5,
                             n_estimators=100)

X_supervised = pd.DataFrame(data=X_train.values[groups_dict.groups[subject_supervised]])
y_supervised = pd.DataFrame(data=y_train.values[groups_dict.groups[subject_supervised]])

X_unlabeled = pd.DataFrame(data=X_train.values[subjects_unsupervised])

# print('super', len(X_supervised))
# print('unsuper', len(X_unlabeled))
# print('all', len(X_train))
model.fit(X_supervised, y_supervised)
y_hat = model.predict(X_test)

print('_______initial supervised model______')
print(classification_report(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
accuracy = balanced_accuracy_score(y_test, y_hat)
print('balanced accuracy', accuracy)

# current_X = X_supervised.copy(deep=True)
# current_y = y_supervised.copy(deep=True)

X_aug = pd.DataFrame(data=X_train.values[groups_dict.groups[subject_supervised]])
y_aug = pd.DataFrame(data=y_train.values[groups_dict.groups[subject_supervised]])

sample_rate = 0.2
batch_size = 1000
i = 0
while i < len(X_unlabeled.values):
    start = i
    end = start + batch_size
    if i >= len(X_unlabeled.values):
        end = len(X_unlabeled.values)-1
        i = end
    else:
        i = end
    # create augmented data
    X_aug, y_aug = create_augmented_data(model, X_aug, y_aug, X_unlabeled.values[start:end], sample_rate)

    # Train new model with augmented training set
    model = RandomForestClassifier(criterion='gini',
                             max_features=None,
                             min_samples_split=5,
                             n_estimators=100)
    
    model.fit(X_aug, y_aug)
    
    # Evaluate new model
    y_hat = model.predict(X_test)
    print('_______new model______')
    print(classification_report(y_test, y_hat))
    print(confusion_matrix(y_test, y_hat))
    accuracy = balanced_accuracy_score(y_test, y_hat)
    print('balanced accuracy', accuracy)
    

/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:106: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______initial supervised model______
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2566
           1       0.87      0.93      0.90      2661
           2       0.72      0.60      0.65       395
           3       0.81      0.52      0.63       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.83      0.74      0.78      6033
weighted avg       0.88      0.88      0.87      6033

[[2369  136   16   45]
 [ 142 2485   30    4]
 [  68   89  236    2]
 [  18  133   47  213]]
balanced accuracy 0.7432006982764872
6093
6093
     cat  probs
500    0    1.0
747    0    1.0
369    0    1.0
367    0    1.0
366    0    1.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______new model______
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2566
           1       0.88      0.94      0.90      2661
           2       0.73      0.61      0.66       395
           3       0.82      0.52      0.64       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.84      0.75      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2373  133   16   44]
 [ 135 2490   34    2]
 [  72   81  241    1]
 [  19  138   39  215]]
balanced accuracy 0.7484412600790409
6293
6293
     cat  probs
873    0    1.0
359    0    1.0
357    0    1.0
355    0    1.0
354    0    1.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

_______new model______
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2566
           1       0.87      0.93      0.90      2661
           2       0.70      0.59      0.64       395
           3       0.82      0.50      0.62       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.82      0.74      0.77      6033
weighted avg       0.87      0.88      0.87      6033

[[2366  144   15   41]
 [ 138 2486   34    3]
 [  70   92  232    1]
 [  20  134   52  205]]
balanced accuracy 0.7356045385911419
7893
7893
     cat  probs
787    0    1.0
431    0    1.0
424    0    1.0
425    0    1.0
426    0    1.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


_______new model______
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      2566
           1       0.87      0.94      0.90      2661
           2       0.71      0.61      0.66       395
           3       0.83      0.51      0.63       411

   micro avg       0.88      0.88      0.88      6033
   macro avg       0.83      0.74      0.78      6033
weighted avg       0.88      0.88      0.88      6033

[[2375  134   17   40]
 [ 132 2491   35    3]
 [  70   84  240    1]
 [  19  140   44  208]]
balanced accuracy 0.7438392465937511
8093
8093
     cat  probs
0      0    1.0
603    0    1.0
605    0    1.0
606    0    1.0
607    0    1.0


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


KeyboardInterrupt: 

In [1]:
# Pseudo-labeling with random selecion for NN
from numpy import mean
from numpy import std
from numpy import dstack
import numpy as np
from pandas import read_csv
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    print('loaded', len(loaded[0]))
    loaded = dstack(loaded)
    print('stacked', loaded.shape)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, freq, win, prefix=''):
    filepath = prefix + group + '/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'_'+win+'_'+freq+'.csv', 'total_acc_y_'+group+'_'+win+'_'+freq+'.csv', 'total_acc_z_'+group+'_'+win+'_'+freq+'.csv']
#     # body acceleration
#     filenames += ['body_acc_x_'+group+'_'+win+'_'+freq+'.csv', 'body_acc_y_'+group+'_'+win+'_'+freq+'.csv', 'body_acc_z_'+group+'_'+win+'_'+freq+'.csv']
#     # body gyroscope
#     filenames += ['body_gyro_x_'+group+'_'+win+'_'+freq+'.csv', 'body_gyro_y_'+group+'_'+win+'_'+freq+'.csv', 'body_gyro_z_'+group+'_'+win+'_'+freq+'.csv']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/state_'+group+'_'+win+'_'+freq+'.csv')
    print('X:', filenames)
    print('y:', prefix + group + '/state_'+group+'_'+win+'_'+freq+'.csv')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(freq, win, prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', freq, win, prefix)
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', freq, win, prefix)
    print(testX.shape, testy.shape)
    # zero-offset class values
#     trainy = trainy - 1
#     testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    y_true = testy
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape, y_true.shape)
    return trainX, trainy, testX, testy, y_true

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [3]:
# lstm model
from sklearn.metrics import confusion_matrix
from keras.layers import Bidirectional

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 15, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    prediction = model.predict_classes(testX)
    return prediction, accuracy

# run an experiment
def run_experiment(freq, win, repeats=1):
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
#     score = evaluate_model(trainX, trainy, testX, testy)
#     print(score)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        pred_classes, score = evaluate_model(trainX, trainy, testX, testy)
        print(score)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
        print(classification_report(y_true, pred_classes))
        print(confusion_matrix(y_true, pred_classes))
    # summarize results
    summarize_results(scores)
    return pred_classes, y_true

# run the experiment
freq = '50'
win = '50'
pred_classes, y_true = run_experiment(freq, win)

loaded 21436
stacked (21436, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(21436, 50, 3) (21436, 1)
loaded 6030
stacked (6030, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(6030, 50, 3) (6030, 1)
(21436, 50, 3) (21436, 4) (6030, 50, 3) (6030, 4) (6030, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
21436/21436 [==============================] - 45s 2ms/step - loss: 1.1410 - categorical_accuracy: 0.4543
Epoch 2/15
 1930/21436 [=>............................] - ETA: 46s - loss: 1.1126 - categorical_accuracy: 0.4731

KeyboardInterrupt: 

In [4]:
from random import sample
import pandas as pd
import numpy as np

# K-fold pseudo-labeling
def create_augmented_set(model, X, y, X_test, sample_rate, target='state'):
    '''
    Create and return the augmented_train set that consists
    of pseudo-labeled and labeled data.
    '''
    num_of_samples = int(len(X_test) * sample_rate)

    # Train the model and create the pseudo-labels
    #model.fit(X, y)
    y_pred = to_categorical(model.predict_classes(X_test))
    
    # Add the pseudo-labels to the test set
#    augmented_test = X_test.copy()
    augmented_X = X.copy()
    augmented_y = y.copy()
    
    # Take a subset of the test set with pseudo-labels and append in onto
    # the training set
    indices = np.random.randint(0, len(X_test), size=(1,num_of_samples))[0]
    augmented_X = np.append(augmented_X, X_test[indices], axis=0)
#     print('aug', augmented_y)
#     print('pred', y_pred)
    augmented_y = np.append(augmented_y, y_pred[indices], axis=0)
    
    return augmented_X, augmented_y

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from keras.layers import Bidirectional
from sklearn.model_selection import KFold
from numpy import argmax
import numpy as np

# run an experiment
def run_experiment(freq, win, repeats=1):
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
    print(len(testX))
    
    # Partially train baseline model
    verbose, epochs, batch_size = 1, 5, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    
    scores = list()
    for train, test in kf.split(trainX):
        print('train', len(train))
        print('test', len(test))
#         y = np.argmax(trainy[test], axis=1, out=None)
        model.fit(trainX[train], trainy[train], epochs=epochs, batch_size=batch_size, verbose=verbose)
        _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
        prediction = model.predict_classes(testX)
        print('WITHOUT Pseudo-label')
        print(classification_report(y_true, prediction))
        print(confusion_matrix(y_true, prediction))
        scores_wo.append(accuracy * 100.0)
        print('Accuracy: %.3f' % accuracy)
        
        # With augmented training set
        sample_rate = 0.2
        augmented_X, augmented_y = create_augmented_train(model, trainX[train], trainy[train], trainX[test], sample_rate, target='state')
        print('aug', len(augmented_X))
        verbose, epochs, batch_size = 1, 5, 10
        n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
        model = Sequential()
        model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(n_outputs, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

        model.fit(augmented_X, augmented_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
        _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
        prediction = model.predict_classes(testX)
        print('WITH Pseudo-label')
        print(classification_report(y_true, prediction))
        print(confusion_matrix(y_true, prediction))
        scores_wo.append(accuracy * 100.0)
        print('Accuracy: %.3f' % accuracy)
    return prediction, y_true

# run the experiment
freq = '50'
win = '50'
pred_classes, y_true = run_experiment(freq, win)

loaded 21436
stacked (21436, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(21436, 50, 3) (21436, 1)
loaded 6030
stacked (6030, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(6030, 50, 3) (6030, 1)
(21436, 50, 3) (21436, 4) (6030, 50, 3) (6030, 4) (6030, 1)
6030


TypeError: split() missing 1 required positional argument: 'X'

In [25]:
# Pseudo-labeling without selective inclusion of samples
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from keras.layers import Bidirectional
from sklearn.model_selection import KFold
from numpy import argmax
import numpy as np

# run an experiment
def run_experiment(freq, win, repeats=1):
    baseline_margin = 10000
    step_size = 1000
    
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
    print(len(testX))
    
    Xtrain = trainX[:baseline_margin]
    ytrain = trainy[:baseline_margin]
    
    print(ytrain)
    # evaluate model function
    verbose, epochs, batch_size = 1, 5, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.fit(trainX[:baseline_margin], trainy[:baseline_margin], validation_split=0.25, class_weight={0: 1, 1: 1, 2: 4, 3: 4}, epochs=epochs, batch_size=batch_size, verbose=verbose)
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    prediction = model.predict_classes(testX)
    print('BASELINE against test set')
    print(classification_report(y_true, prediction))
    print(confusion_matrix(y_true, prediction))
    print('Accuracy: %.3f' % accuracy)
    
    scores = list()
    i = 0
    while i < len(trainX):
        print('i:', i)
        
        # Create pseudo-labeled training set
        new_train_X = trainX[i:i+step_size]
        y = np.argmax(trainy[i:i+step_size], axis=1, out=None)
        _, accuracy = model.evaluate(trainX[i:i+step_size], trainy[i:i+step_size], batch_size=batch_size, verbose=1)
        prediction = model.predict_classes(trainX[i:i+step_size])
        print('Non-augmented model against new training data')
        print(classification_report(y, prediction))
        print(confusion_matrix(y, prediction))   
        print('Accuracy: %.3f' % accuracy)
        
        # Retrain with new data
        print('Augmented model against new training data')
        y_cat = to_categorical(prediction, num_classes=4)
        model.train_on_batch(new_train_X, y_cat, class_weight={0: 1, 1: 1, 2: 4, 3: 4})
        _, accuracy = model.evaluate(new_train_X, trainy[i:i+step_size], batch_size=batch_size, verbose=1)
        pred_aug = model.predict_classes(new_train_X, batch_size=batch_size, verbose=verbose)
        print(classification_report(y, pred_aug))
        print(confusion_matrix(y, pred_aug))
        print('Accuracy: %.3f' % accuracy)
        
        # Retrain with new data
        print('Augmented model against test set')
        _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
        pred_test = model.predict_classes(testX)
        print(classification_report(y_true, pred_test))
        print(confusion_matrix(y_true, pred_test))
        print('Accuracy: %.3f' % accuracy)

        i += step_size
    return prediction, y_true

# run the experiment
freq = '50'
win = '50'
pred_classes, y_true = run_experiment(freq, win)

loaded 21436
stacked (21436, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(21436, 50, 3) (21436, 1)
loaded 6030
stacked (6030, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(6030, 50, 3) (6030, 1)
(21436, 50, 3) (21436, 4) (6030, 50, 3) (6030, 4) (6030, 1)
6030
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]
Train on 7500 samples, validate on 2500 samples
Epoch 1/5
7500/7500 [==============================] - 18s 2ms/step - loss: 2.0883 - categorical_accuracy: 0.4084 - val_loss: 2.5154 - val_categorical_accuracy: 0.3544
Epoch 2/5
7500/7500 [==============================] - 16s 2ms/step - loss: 2.0808 - categorical_accuracy: 0.4213 - val_loss: 2.5159 - val_categorical_accuracy: 0.3652
Epoch 3/5
7500/7500 [===========================

KeyboardInterrupt: 

In [14]:
# Pseudo-labeling with selective inclusion of samples with highest probability
# With separate validation set for during training
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from keras.layers import Bidirectional
from sklearn.model_selection import KFold
from numpy import argmax
import numpy as np
from keras.callbacks import History 
history = History()

# run an experiment
def run_experiment(freq, win, repeats=1):
    baseline_margin = 18000
    val_margin = 0
    step_size = 1500
    
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
    print(len(testX))
    
    Xtrain = trainX[:baseline_margin]
    ytrain = trainy[:baseline_margin]
    
    Xval = trainX[baseline_margin:baseline_margin+val_margin]
    yval = trainy[baseline_margin:baseline_margin+val_margin]
    
    # evaluate model function
    verbose, epochs, batch_size = 1, 15, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.fit(trainX[:baseline_margin], trainy[:baseline_margin], validation_split=0.25, class_weight={0: 1, 1: 1, 2: 4, 3: 4}, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[history])
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    prediction = model.predict_classes(testX)
    print('BASELINE against test set')
    print(classification_report(y_true, prediction))
    print(confusion_matrix(y_true, prediction))
    print('Accuracy: %.3f' % accuracy)
#     print('\nHISTORY:', history.history)
#     print('WEIGHTS:', len(model.get_weights()))
    
    scores = list()
    i = baseline_margin + val_margin
    cnt = 1
    while i < len(trainX):
        print('i:', i)
        
        # Create pseudo-labeled training set
        new_train_X = trainX[i:i+step_size]
        y = np.argmax(trainy[i:i+step_size], axis=1, out=None)
        _, accuracy = model.evaluate(trainX[i:i+step_size], trainy[i:i+step_size], batch_size=batch_size, verbose=1)
        prediction = model.predict_classes(trainX[i:i+step_size])
        print(model.predict_proba(trainX[i:i+step_size]))
        print('Non-augmented model against new training data')
        print(classification_report(y, prediction))
        print(confusion_matrix(y, prediction))   
        print('Accuracy: %.3f' % accuracy)
        
        # Retrain with new data
        print('Augmented model against new training data')
        y_cat = to_categorical(prediction, num_classes=4)
#        model.train_on_batch(new_train_X, y_cat, class_weight={0: 1, 1: 1, 2: 4, 3: 4})
        model.fit(new_train_X, y_cat, class_weight={0: 1, 1: 1, 2: 4, 3: 4}, initial_epoch=epochs*cnt, epochs=epochs*cnt+epochs)
#         print('\nHISTORY:', history.history)
#         print('WEIGHTS:', len(model.get_weights()))
#        model.fit(new_train_X, y_cat, epocwhaths=epochs, class_weight={0: 1, 1: 1, 2: 4, 3: 4}, validation_data=(Xval, yval))
        _, accuracy = model.evaluate(new_train_X, trainy[i:i+step_size], batch_size=batch_size, verbose=1)
        pred_aug = model.predict_classes(new_train_X, batch_size=batch_size, verbose=verbose)
        print(classification_report(y, pred_aug))
        print(confusion_matrix(y, pred_aug))
        print('Accuracy: %.3f' % accuracy)
        
        # Retrain with new data
        print('Augmented model against test set')
        _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
        pred_test = model.predict_classes(testX)
        print(classification_report(y_true, pred_test))
        print(confusion_matrix(y_true, pred_test))
        print('Accuracy: %.3f' % accuracy)

        i += step_size
        cnt += 1
    return prediction, y_true

# run the experiment
freq = '50'
win = '50'
pred_classes, y_true = run_experiment(freq, win)

loaded 21436
stacked (21436, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(21436, 50, 3) (21436, 1)
loaded 6030
stacked (6030, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(6030, 50, 3) (6030, 1)
(21436, 50, 3) (21436, 4) (6030, 50, 3) (6030, 4) (6030, 1)
6030
Train on 13500 samples, validate on 4500 samples
Epoch 1/15
13500/13500 [==============================] - 42s 3ms/step - loss: 2.1732 - categorical_accuracy: 0.3350 - val_loss: 1.9480 - val_categorical_accuracy: 0.3802
Epoch 2/15
13500/13500 [==============================] - 45s 3ms/step - loss: 2.1593 - categorical_accuracy: 0.3664 - val_loss: 1.9412 - val_categorical_accuracy: 0.5909
Epoch 3/15
13500/13500 [==============================] - 55s 4ms/step - loss: 2.1519 - categorical_accuracy: 0.3737 - val_loss: 1.8604 -

1500/1500 [==============================] - 2s 2ms/step - loss: 0.4875 - categorical_accuracy: 0.9233
Epoch 32/45
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4525 - categorical_accuracy: 0.9127
Epoch 33/45
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3075 - categorical_accuracy: 0.9473
Epoch 34/45
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2927 - categorical_accuracy: 0.9587
Epoch 35/45
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2740 - categorical_accuracy: 0.9573
Epoch 36/45
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2470 - categorical_accuracy: 0.9620
Epoch 37/45
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1975 - categorical_accuracy: 0.9707: 0s - loss: 0.1924 - categorical_ac
Epoch 38/45
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2081 - categorical_accuracy: 0.9627
Epoch 39/45
1500/1500 [==============================] -